# import pacakge and set up environment

In [1]:
import requests
import json
import os
from langchain.text_splitter import CharacterTextSplitter
import faiss
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
import pickle
os.environ["OPENAI_API_BASE"] = "https://api.chatanywhere.tech/v1"
os.environ["OPENAI_API_KEY"] = "sk-X6qmvKSqTxaeFIrmq9v2KlAN6QJEKwxi6eGX5ads7wYERbx0"


# Loading data from Strikingly 

In [2]:

r = requests.get("https://support.strikingly.com/api/v2/help_center/en-us/articles.json")
articles_info = json.loads(r.text)['articles']
pages = []
from bs4 import BeautifulSoup

def extract_text_from(html):
    soup = BeautifulSoup(html, features="html.parser")
    text = soup.get_text()

    lines = (line.strip() for line in text.splitlines())
    return '\n'.join(line for line in lines if line)
for article in articles_info:

    pages.append({'text': extract_text_from(article['body']), 'source': article['html_url']})
    
    

# Process data

In [3]:

text_splitter = CharacterTextSplitter(chunk_size=1500, separator="\n")
docs, metadatas = [], []
for page in pages:
    splits = text_splitter.split_text(page['text'])
    docs.extend(splits)
    metadatas.extend([{"source": page['source']}] * len(splits))
    print(f"Split {page['source']} into {len(splits)} chunks")


store = FAISS.from_texts(docs, OpenAIEmbeddings(), metadatas=metadatas)
with open("faiss_store.pkl", "wb") as f:
    pickle.dump(store, f)

Split https://support.strikingly.com/hc/en-us/articles/33571866019995-Adding-Images-to-Your-Website into 3 chunks
Split https://support.strikingly.com/hc/en-us/articles/33195129962651-Strikingly-s-AI-Logo-Maker into 2 chunks
Split https://support.strikingly.com/hc/en-us/articles/32500484566811-Strikingly-AI-Site-Builder into 3 chunks
Split https://support.strikingly.com/hc/en-us/articles/30097621863835-Strikingly-Email-Creating-Filters-on-Webmail into 1 chunks
Split https://support.strikingly.com/hc/en-us/articles/25888063899931-How-do-I-delete-a-form into 1 chunks
Split https://support.strikingly.com/hc/en-us/articles/25585768352667-Can-I-upload-a-pdf-file-document-for-my-visitors-to-download into 1 chunks
Split https://support.strikingly.com/hc/en-us/articles/25511392983195-Can-my-website-carry-my-name into 1 chunks
Split https://support.strikingly.com/hc/en-us/articles/25511191343899-How-can-I-forward-redirect-my-domain-from-another-registrar-to-my-Strikingly-domain into 1 chunks
Sp

# Chat

In [4]:
import faiss
from langchain import OpenAI
from langchain.chains import VectorDBQAWithSourcesChain
from langchain.chat_models import ChatOpenAI 
import pickle
import argparse
import os

# parser = argparse.ArgumentParser(description='Paepper.com Q&A')
# parser.add_argument('question', type=str, help='Your question for Paepper.com')
# args = parser.parse_args()

with open("faiss_store.pkl", "rb") as f:
    store = pickle.load(f)

chain = VectorDBQAWithSourcesChain.from_llm(
    llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"),  # 使用ChatOpenAI替代OpenAI
    vectorstore=store
)
result = chain({"question": 'How to add images to your website'})

print(f"Answer: {result['answer']}")
print(f"Sources: {result['sources']}")


/Users/mz/anaconda3/lib/python3.11/site-packages/langchain/chains/qa_with_sources/vector_db.py:66: UserWarning: `VectorDBQAWithSourcesChain` is deprecated - please use `from langchain.chains import RetrievalQAWithSourcesChain`
  warnings.warn(


Answer: To add images to your website, you can either click on the "Background" option in the Site Editor or use the "Manage Gallery" feature. Then proceed to upload the desired image and make necessary adjustments for positioning and scale. Additionally, use the "Image & Video" sections in the Site Editor or click on "Add New Section" to insert images. 

Sources: https://support.strikingly.com/hc/en-us/articles/33571866019995-Adding-Images-to-Your-Website, https://support.strikingly.com/hc/en-us/articles/33571866019995-Adding-Images-to-Your-Website, https://support.strikingly.com/hc/en-us/articles/33571866019995-Adding-Images-to-Your-Website, https://support.strikingly.com/hc/en-us/articles/25409142989851-How-to-build-a-website
